In [12]:
"""# 언어별 분류 작업
- 정확도가 높은 fasttext 모듈로 분류
"""
!pip install fasttext
import copy
import fasttext
import pandas as pd
import re

In [17]:
def preprocessing(name):
        
    path = '/content/drive/MyDrive/Colab Notebooks/[공유] Mulcam_Army 공유폴더!/크롤링 한 자료/youtube/영상별 댓글/'
    comment_file = f'comments_youtube_{name}.csv'     #aespa
    df = pd.read_csv(path+comment_file, encoding='utf-8', header=None)
    df.columns=['comment','like']
    
    # 중복 값 제거
    print(f"{name} 전처리 시작")
    print('\n')
    print('중복 제거 전 :',df.shape)
    df = df.drop_duplicates(['comment'],keep='last',ignore_index=True)
    print(f'{name} 중복 제거 후 :',df.shape)
    print('\n')

    # 소문자로 바꾸기
    df['comment'] = df['comment'].str.lower()
    
    # 전처리 전 원본 보존
    copy_data = copy.deepcopy(df)
    
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags=re.UNICODE)

    #분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
    han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')

    comment_result = []

    for i in copy_data['comment'].values:
        tokens = re.sub(emoji_pattern,"",i)
        tokens = re.sub(han,"",tokens)
        comment_result.append(tokens)

    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

    punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

    def clean_punc(text, punct, mapping):
        for p in mapping:
            text = text.replace(p, mapping[p])
        
        for p in punct:
            text = text.replace(p, f' {p} ')
        
        specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
        for s in specials:
            text = text.replace(s, specials[s])
        
        return text.strip()

    cleaned_corpus = []
    for sent in comment_result:
        cleaned_corpus.append(clean_punc(sent, punct, punct_mapping))



    def clean_text(texts):
        corpus = []
        for i in range(0, len(texts)):
            review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
            review = re.sub(r'\d+','', str(texts[i]))# remove number
            review = review.lower() #lower case
            review = re.sub(r'\s+', ' ', review) #remove extra space
            review = re.sub(r'<[^>]+>','',review) #remove Html tags
            review = re.sub(r'\s+', ' ', review) #remove spaces
            review = re.sub(r"^\s+", '', review) #remove space from start
            review = re.sub(r'\s+$', '', review) #remove space from the end
            corpus.append(review)
        return corpus

    basic_preprocessed_corpus = clean_text(cleaned_corpus)
    comment_result = pd.DataFrame(basic_preprocessed_corpus, columns=["comment"])

    model = fasttext.load_model('/content/drive/MyDrive/Colab Notebooks/[공유] Mulcam_Army 공유폴더!/lid.176.ftz')

    predict = []
    for t in comment_result.comment.values:
        predict.append(model.predict(t,k=1))

    ty = pd.DataFrame(predict)

    comment = []
    for num, txt in enumerate(ty[0]):
        txt = str(txt)
        if txt == "('__label__ko',)":
            b = re.sub(txt,"ko",txt)
            comment.append(b)
        elif txt == "('__label__en',)":
            b = re.sub(txt,"en",txt)
            comment.append(b)
        elif txt == "('__label__id',)":
            b = re.sub(txt,"id",txt)
            comment.append(b)
        elif txt == "('__label__es',)":
            b = re.sub(txt,"es",txt)
            comment.append(b)
        else:
            b = re.sub(txt,"etc",txt)
            comment.append(b)

    comment = pd.DataFrame(comment)
    print('\n')
    print(f'{name} 댓글 언어 구성')
    print(comment.value_counts())
    print('\n')
    pd.set_option('max_columns',50)
    pd.set_option('max_rows',100)
    # ty_sum.to_csv('ty_sum.csv', encoding='cp949')


    like = pd.DataFrame(copy_data['like'])
    data = pd.concat([comment_result,like, comment],axis=1)
    data.columns = ['comment','like','lang']


    path_preprocess='/content/drive/MyDrive/Colab Notebooks/[공유] Mulcam_Army 공유폴더!/크롤링 한 자료/youtube/preprocesing_comment/'
    data.to_csv(path_preprocess+'prepro_'+comment_file,  encoding='utf-8', header=None, index=None)

    print(data[:2])
    print(f"{name} 전처리 끝")
    print('\n')

    return data

In [18]:
artist_name=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/[공유] Mulcam_Army 공유폴더!/아티스트이름 - Sheet1.csv')
name=artist_name['engName']
name.dropna(inplace=True)
name=name.reset_index(drop=True)
name.drop([3,5,24,29,37,38,48],axis=0,inplace=True)
name=name.reset_index(drop=True)
name[:4]

0            GOT7
1    GOLDEN CHILD
2    KIM SE JEONG
3          NU_EST
Name: engName, dtype: object

In [19]:
for i in range(len(name)):
    preprocessing(name[i])

GOT7 전처리 시작


중복 제거 전 : (26, 2)
GOT7 중복 제거 후 : (26, 2)




GOT7 댓글 언어 구성
(en)     19
(ko)      6
(etc)     1
dtype: int64


                                             comment  like  lang
0  wish all the best for this boy he deserves mor...   255  (en)
1  can ' t believe how underrated it is and how m...    27  (en)
GOT7 전처리 끝


GOLDEN CHILD 전처리 시작


중복 제거 전 : (19644, 2)
GOLDEN CHILD 중복 제거 후 : (16543, 2)






GOLDEN CHILD 댓글 언어 구성
(en)     9904
(es)     3680
(etc)    2816
(id)       96
(ko)       47
dtype: int64


                                             comment  like   lang
0  yesterday is a historytomorrow is a mysterytod...   431   (en)
1  lisa - firejennie - fabrose - fiercejisoo - fancy  5.4만  (etc)
GOLDEN CHILD 전처리 끝


KIM SE JEONG 전처리 시작


중복 제거 전 : (1892, 2)
KIM SE JEONG 중복 제거 후 : (1851, 2)






KIM SE JEONG 댓글 언어 구성
(en)     1032
(ko)      634
(etc)     134
(id)       35
(es)       16
dtype: int64


                              comment  like  lang
0            i love ur voice sejeong🥺   672  (en)
1  와 진짜 목소리 대박인데근데 양손은 서로 가위바위보하는 건가요  1.3천  (ko)
KIM SE JEONG 전처리 끝


NU_EST 전처리 시작


중복 제거 전 : (501, 2)
NU_EST 중복 제거 후 : (498, 2)




NU_EST 댓글 언어 구성
(en)     212
(ko)     191
(etc)     74
(es)      18
(id)       3
dtype: int64


                                             comment  like  lang
0  i can ' t believe something as simple as this ...    86  (en)
1  they are in their th year yet they are still t...   179  (en)
NU_EST 전처리 끝


DAY6 전처리 시작


중복 제거 전 : (18875, 2)
DAY6 중복 제거 후 : (15879, 2)






DAY6 댓글 언어 구성
(en)     9376
(es)     3563
(etc)    2798
(id)       97
(ko)       45
dtype: int64


                                             comment  like   lang
0  lisa - firejennie - fabrose - fiercejisoo - fancy  5.4만  (etc)
1  i really hope blackpink will be able to perfor...  1.3만   (en)
DAY6 전처리 끝


RED VELVET 전처리 시작


중복 제거 전 : (2336, 2)
RED VELVET 중복 제거 후 : (2149, 2)






RED VELVET 댓글 언어 구성
(en)     1643
(etc)     247
(ko)      174
(id)       48
(es)       37
dtype: int64


                                             comment  like  lang
0  it feels like a dream that we ' re getting all...  1,6K  (en)
1            drop the queendom dance practice bestie    3K  (en)
RED VELVET 전처리 끝


MAMAMOO 전처리 시작


중복 제거 전 : (8594, 2)
MAMAMOO 중복 제거 후 : (8293, 2)




...

                                             comment  like  lang
0  ways to enjoy kcon : tact hi even more http : ...     8  (en)
1          shinwon ' s part is only made for shinwon   139  (en)
PENTAGON 전처리 끝


PURPLE KISS 전처리 시작


중복 제거 전 : (487, 2)
PURPLE KISS 중복 제거 후 : (484, 2)




PURPLE KISS 댓글 언어 구성
(en)     390
(etc)     49
(ko)      25
(es)      19
(id)       1
dtype: int64




                                             comment  like  lang
0  if y ' all wanna cut cut the lalala part stop ...  1548  (en)
1  yuki ' s outfit is simple but it looks really ...   480  (en)
PURPLE KISS 전처리 끝


fromis_9 전처리 시작


중복 제거 전 : (19923, 2)
fromis_9 중복 제거 후 : (16764, 2)






fromis_9 댓글 언어 구성
(en)     10476
(es)      3421
(etc)     2736
(id)        84
(ko)        47
dtype: int64


                                             comment  like   lang
0  jisoo : best visualjennie : best rapperrosé : ...    1천  (etc)
1  any one else listening this song again and aga...  1.6천   (en)
fromis_9 전처리 끝


HA SUNG WOON 전처리 시작


중복 제거 전 : (381, 2)
HA SUNG WOON 중복 제거 후 : (377, 2)




HA SUNG WOON 댓글 언어 구성
(en)     173
(ko)     156
(etc)     33
(es)      12
(id)       3
dtype: int64




                                             comment  like  lang
0  wish all the best for this boy he deserves mor...   255  (en)
1  can ' t believe how underrated it is and how m...    27  (en)
HA SUNG WOON 전처리 끝


HIGHLIGHT 전처리 시작


중복 제거 전 : (574, 2)
HIGHLIGHT 중복 제거 후 : (570, 2)




HIGHLIGHT 댓글 언어 구성
(ko)     407
(en)     130
(etc)     22
(es)       6
(id)       5
dtype: int64




                                             comment  like  lang
0  와 이런 비하인드 처음이라 너무 좋아서 무슨 말을 해야할지 모르겠지만 사랑합니다 또...   206  (ko)
1  나는 이런 도란도란하고 귀여운 하이라이트와 본업천재 하이라이트가 보여지는 영상이 너...   148  (ko)
HIGHLIGHT 전처리 끝


